In [21]:
import requests
import json
import os
import re
import time
from googlesearch import search
from bs4 import BeautifulSoup as bs
from joblib import Parallel, delayed

In [84]:
request_timeout = 10
headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:49.0) Gecko/20100101 Firefox/49.0',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
        'Accept-Language': 'en-US,en;q=0.5',
        'Accept-Encoding': 'gzip, deflate',
        'Connection': 'keep-alive',
        'Cookie': '1P_JAR=2023-10-25-12; AEC=Ackid1QRp0k00ZR8wkSchjrI4TxT67QDIsxkdP6qfLqh3WuD9-nfTpqc0-c; NID=511=LP-_NqTCw8FjacALQJp9P_dHSzqHtAlaG48GObc-26iQjj8wHcIcPsxGbzizVB43rpKlKmpN72SYIJtjgQMr0VJjaqQ5-eA8Oi1MGuZJjsV3dMCyhdWYjwPKVMabTC-Dez3kV48ORewEX5O9jJdCItsjUaENbNTbqyyVot6QuNc',
        'Cache-Control': 'private, max-age=0',
        'Set-Cookie': '1P_JAR=2023-10-25-12; expires=Fri, 24-Nov-2023 12:24:37 GMT; path=/; domain=.google.com; Secure; SameSite=none'
}
html_pattern = "<(?:\"[^\"]*\"['\"]*|'[^']*'['\"]*|[^'\">])+>"

#search variables
from_year = 2013
to_year = 2022
total_news_per_year = 100

In [85]:
"""
def get_google_news_urls(q, page, from_year, to_year):
    urls = []
    queries_tag = "Gx5Zad fP1Qef xpd EtOod pkphOe"
    title_tag = "BNeawe vvjwJb AP7Wnd"
    search_url = f'https://www.google.com/search?q={q}&lr=lang_en&cd_min=1/1/2015&cd_max=2013&tbs=sbd:1,lr:lang_en,cdr:1,cd_min:1/1/2015,cd_max:2015&tbm=nws&start={page * 10}'
    print(search_url)
    search_result = requests.get(search_url)
    soup = bs(search_result.content, "html.parser")
    body = soup.find(id="main")
    queries = body.find_all("div", queries_tag)
    for i in queries:
        output = {}
        header = i.find('a')
        title = header.find('div', 'BNeawe vvjwJb AP7Wnd').text
        url = header['href']
        url = url.replace('/url?q=', '')
        url = url[0: s.index("&sa")]
        output['url'] = url
        output['title'] = title
        urls.append(output)    
        
    return urls
"""

'\ndef get_google_news_urls(q, page, from_year, to_year):\n    urls = []\n    queries_tag = "Gx5Zad fP1Qef xpd EtOod pkphOe"\n    title_tag = "BNeawe vvjwJb AP7Wnd"\n    search_url = f\'https://www.google.com/search?q={q}&lr=lang_en&cd_min=1/1/2015&cd_max=2013&tbs=sbd:1,lr:lang_en,cdr:1,cd_min:1/1/2015,cd_max:2015&tbm=nws&start={page * 10}\'\n    print(search_url)\n    search_result = requests.get(search_url)\n    soup = bs(search_result.content, "html.parser")\n    body = soup.find(id="main")\n    queries = body.find_all("div", queries_tag)\n    for i in queries:\n        output = {}\n        header = i.find(\'a\')\n        title = header.find(\'div\', \'BNeawe vvjwJb AP7Wnd\').text\n        url = header[\'href\']\n        url = url.replace(\'/url?q=\', \'\')\n        url = url[0: s.index("&sa")]\n        output[\'url\'] = url\n        output[\'title\'] = title\n        urls.append(output)    \n        \n    return urls\n'

In [86]:
def get_google_news_urls(q, total, from_year, to_year):
    urls = []
    for url in search(q, tld="co.in", start=0, stop=total, num=50, pause=1, lang='en', tbs=f'sbd:1,lr:lang_en,cdr:1,cd_min:1/1/{from_year},cd_max:{to_year}', extra_params={'tbm': 'nws'}):
        urls.append(url)
    return urls

In [87]:
def get_news_data(url):
    output = {}
    text = ""
    try:
        article = requests.get(url, headers=headers, timeout=request_timeout)
        status_code = article.status_code
        if status_code >= 400:
            print(url, status_code)
            return None
        soup = bs(article.content, "html.parser")
        datetime = soup.find('meta', property='article:published_time')
        paragraphs = soup.find_all("p")
        title = soup.find('title').text
        if paragraphs is not None:
            for p in paragraphs:
                text += re.sub(html_pattern, '', p.text).strip() + ' '
        
        output['text'] = text
        output['title'] = title
        if datetime is not None:
            output['datetime'] = datetime['content']
        output['url'] = url
        return output
    except requests.exceptions.RequestException as e:
        msg = e

In [88]:
def save_dataset(q, urls, year):
    #urls = get_google_news_urls(q, total, from_year, to_year)
    total_count = 0
    for url in urls:
        try:
            data = get_news_data(url)
            if data is None:
                continue
            if q not in os.listdir():
                cwd = os.getcwd()
                os.mkdir(f'{cwd}/{q}')
            total_count += 1
            file_error_symbols = []
            filename = re.sub(r'[/|:?<>"]', '', data["title"]).strip().replace('\n', '').replace('\r', '')
            f = open(f'./{q}/{filename}.json', "w")
            f.write(json.dumps(data, indent = 4))
            f.close()
        except requests.exceptions.RequestException as e:
            print(e)
    print(f'Saved {total_count} news data in year {year}')
    #print(os.system("npx prettier -w ./dataset/*.json"))

In [89]:
def save_news_urls(q, total, from_year, to_year):
    urls = {}
    for year in range(from_year, to_year + 1):
        urls[str(year)] = get_google_news_urls(q, total, year, year)
        print(f'{year} Done')
        #delay request to prevent 429 Too many Request
        time.sleep(20)
    f = open(f'{q}.json', 'w')
    f.write(json.dumps(urls, indent = 4))
    f.close()
    return urls

In [90]:
def get_urls_from_file(filename):
    f = open(filename, "r")
    data = json.load(f)
    f.close()
    return data

In [91]:
def get_keywords_from_file(filename):
    keywords = []
    f = open(filename, "r")
    for keyword in f:
        keywords.append(keyword)
    f.close()
    return list(map(lambda keyword: keyword.strip(), keywords))

In [92]:
def parallel_save_dataset(q, urls):
    start = time.time()
    Parallel(n_jobs=4)(delayed(save_dataset)(q, urls[year], year) for year in urls)
    end = time.time()
    print('{:.2f} seconds used'.format(end - start))

In [93]:
#run all functions to save news data
def scrap_news_data(total, from_year, to_year, keywords_filename="keywords.txt"):
    #step 1 - load keywords file
    keywords = get_keywords_from_file(keywords_filename)
    for query in keywords:
        print(f'Scrapping {query} ...')
        #step 2 - save urls to json file
        save_news_urls(query, total, from_year, to_year)
        #step 3 - get urls object from previous saved json file
        urls = get_urls_from_file(f'{query}.json')
        #step 4 - web scrapping news data from specific year of urls & save into directory seperately
        parallel_save_dataset(query, urls)

In [94]:
#scrap_news_data(total_news_per_year, from_year, to_year)

In [ ]:
#index,datetime,title,url,text